<a href="https://colab.research.google.com/github/alirezafarhadi01/DrugDesignCourse-FinalProject/blob/main/DeepDTA_Version_4_(Dence).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.metrics import mean_squared_error
import os
import warnings
warnings.filterwarnings('ignore')

# Install and import tqdm for progress bars
try:
    from tqdm import tqdm
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tqdm"])
    from tqdm import tqdm

# Basic optimizations
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['OMP_NUM_THREADS'] = str(os.cpu_count())

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Constants
KIBA_MAX_DRUG_LEN = 100
KIBA_MAX_PROT_LEN = 1000
EMBEDDING_DIM = 128
KIBA_DATA_PATH = './kiba'
SPLIT_PATH = '/content/New Data/kiba_split.json'

# Enable XLA for faster computation
try:
    tf.config.optimizer.set_jit(True)
except:
    pass

def concordance_index(y_true, y_pred):
    """Optimized concordance index calculation."""
    y_true = y_true.flatten().astype(np.float32)
    y_pred = y_pred.flatten().astype(np.float32)

    mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    y_true = y_true[mask]
    y_pred = y_pred[mask]

    if len(y_true) < 2:
        return 0.0

    if len(y_true) > 10000:
        indices = np.random.choice(len(y_true), 10000, replace=False)
        y_true = y_true[indices]
        y_pred = y_pred[indices]

    n = len(y_true)
    indices = np.arange(n)
    i_indices, j_indices = np.meshgrid(indices, indices, indexing='ij')
    mask = i_indices < j_indices

    y_true_i = y_true[i_indices[mask]]
    y_true_j = y_true[j_indices[mask]]
    y_pred_i = y_pred[i_indices[mask]]
    y_pred_j = y_pred[j_indices[mask]]

    different_mask = y_true_i != y_true_j
    if np.sum(different_mask) == 0:
        return 0.0

    y_true_i = y_true_i[different_mask]
    y_true_j = y_true_j[different_mask]
    y_pred_i = y_pred_i[different_mask]
    y_pred_j = y_pred_j[different_mask]

    concordant = ((y_true_i > y_true_j) & (y_pred_i > y_pred_j)) | \
                 ((y_true_i < y_true_j) & (y_pred_i < y_pred_j))
    ties = (y_pred_i == y_pred_j)

    return (np.sum(concordant) + 0.5 * np.sum(ties)) / len(y_true_i)

def create_char_vocab(sequences_dict):
    """Create character vocabulary with progress tracking."""
    char_set = set()
    for seq in tqdm(sequences_dict.values(), desc="Processing sequences", unit="seq"):
        char_set.update(seq)
    chars = sorted(list(char_set))
    char_to_int = {char: idx + 1 for idx, char in enumerate(chars)}
    return char_to_int

def encode_sequences_batch(sequences, char_vocab, max_length, desc="Encoding sequences"):
    """Batch encode sequences with progress bar."""
    encoded_sequences = []
    for sequence in tqdm(sequences, desc=desc, unit="seq"):
        encoded = [char_vocab.get(char, 0) for char in sequence]
        if len(encoded) < max_length:
            encoded += [0] * (max_length - len(encoded))
        else:
            encoded = encoded[:max_length]
        encoded_sequences.append(encoded)
    return np.array(encoded_sequences, dtype=np.int32)

class OptimizedKibaDataProcessor:
    """Enhanced data processor with detailed progress tracking."""

    def __init__(self, data_path='./kiba', max_drug_len=100, max_prot_len=1000):
        self.data_path = data_path
        self.max_drug_len = max_drug_len
        self.max_prot_len = max_prot_len
        self.drug_vocab = None
        self.prot_vocab = None

    def load_data(self):
        """Load data with progress tracking (uses precomputed split from SPLIT_PATH)."""
        try:
            ligands_path = os.path.join(self.data_path, 'ligands_can.txt')
            proteins_path = os.path.join(self.data_path, 'proteins.txt')

            with open(ligands_path, 'r') as f:
                drugs_dict = json.load(f)
            with open(proteins_path, 'r') as f:
                proteins_dict = json.load(f)

            y_file_path = os.path.join(self.data_path, 'Y')
            if os.path.exists(y_file_path):
                try:
                    with open(y_file_path, 'rb') as f:
                        affinity_matrix = pickle.load(f)
                except:
                    with open(y_file_path, 'rb') as f:
                        affinity_matrix = pickle.load(f, encoding='latin1')
            else:
                raise FileNotFoundError(f"Y file not found at {y_file_path}")

            valid_indices = np.where(~np.isnan(affinity_matrix))
            total_valid_pairs = len(valid_indices[0])

            if not os.path.exists(SPLIT_PATH):
                raise FileNotFoundError(f"Split file not found at {SPLIT_PATH}")
            with open(SPLIT_PATH, "r") as f:
                split = json.load(f)
            train_indices = np.array(split["train_indices"], dtype=int)
            test_indices  = np.array(split["test_indices"], dtype=int)

            if "total_valid_pairs" in split and int(split["total_valid_pairs"]) != int(total_valid_pairs):
                print(f"[warn] total_valid_pairs in split ({split['total_valid_pairs']}) != current ({total_valid_pairs})")

            return drugs_dict, proteins_dict, affinity_matrix, train_indices, test_indices, valid_indices

        except Exception as e:
            return None

    def process_data(self):
        """Process data with comprehensive progress tracking."""
        data = self.load_data()
        if data is None:
            return None

        drugs_dict, proteins_dict, affinity_matrix, train_indices, test_indices, valid_indices = data

        self.drug_vocab = create_char_vocab(drugs_dict)
        self.prot_vocab = create_char_vocab(proteins_dict)

        drug_ids = list(drugs_dict.keys())
        protein_ids = list(proteins_dict.keys())

        def create_dataset_optimized(indices, dataset_name):
            """Create dataset with detailed progress tracking."""
            drug_sequences = []
            prot_sequences = []
            affinities = []

            chunk_size = 10000
            total_chunks = (len(indices) + chunk_size - 1) // chunk_size

            chunk_pbar = tqdm(total=total_chunks, desc=f"Processing {dataset_name} chunks", unit="chunk")

            for chunk_idx, i in enumerate(range(0, len(indices), chunk_size)):
                chunk_indices = indices[i:i + chunk_size]
                chunk_drugs = []
                chunk_prots = []
                chunk_affs = []

                sample_pbar = tqdm(
                    chunk_indices,
                    desc=f"Chunk {chunk_idx + 1}/{total_chunks}",
                    unit="sample",
                    leave=False
                )

                for idx in sample_pbar:
                    if idx < len(valid_indices[0]):
                        drug_idx = valid_indices[0][idx]
                        prot_idx = valid_indices[1][idx]

                        if drug_idx < len(drug_ids) and prot_idx < len(protein_ids):
                            drug_id = drug_ids[drug_idx]
                            prot_id = protein_ids[prot_idx]

                            chunk_drugs.append(drugs_dict[drug_id])
                            chunk_prots.append(proteins_dict[prot_id])
                            chunk_affs.append(affinity_matrix[drug_idx, prot_idx])

                sample_pbar.close()

                if chunk_drugs:
                    encoded_drugs = encode_sequences_batch(
                        chunk_drugs, self.drug_vocab, self.max_drug_len,
                        desc=f"Encoding drugs (chunk {chunk_idx + 1})"
                    )
                    encoded_prots = encode_sequences_batch(
                        chunk_prots, self.prot_vocab, self.max_prot_len,
                        desc=f"Encoding proteins (chunk {chunk_idx + 1})"
                    )

                    drug_sequences.append(encoded_drugs)
                    prot_sequences.append(encoded_prots)
                    affinities.extend(chunk_affs)

                chunk_pbar.update(1)

            chunk_pbar.close()

            if drug_sequences:
                drugs = np.concatenate(drug_sequences, axis=0)
                proteins = np.concatenate(prot_sequences, axis=0)
                affinities = np.array(affinities, dtype=np.float32)
            else:
                drugs = np.array([], dtype=np.int32).reshape(0, self.max_drug_len)
                proteins = np.array([], dtype=np.int32).reshape(0, self.max_prot_len)
                affinities = np.array([], dtype=np.float32)

            return drugs, proteins, affinities

        XD_train, XT_train, y_train = create_dataset_optimized(train_indices, "training")
        XD_test, XT_test, y_test = create_dataset_optimized(test_indices, "test")

        return ((XD_train, XT_train, y_train),
                (XD_test, XT_test, y_test),
                (len(self.drug_vocab) + 1, len(self.prot_vocab) + 1))

def build_improved_deepdta_model(drug_vocab_size, prot_vocab_size, max_drug_len, max_prot_len):
    """
    Builds a simplified model using only Dense layers to test the hypothesis
    that the CNN complexity in DeepDTA is not necessary.

    This architecture flattens the embedded sequences and feeds them into a
    series of Dense, BatchNormalization, and Dropout layers.
    """
    EMBEDDING_DIM = 128
    DENSE_UNITS_1 = 1024
    DENSE_UNITS_2 = 512
    DROPOUT_RATE = 0.2
    LEARNING_RATE = 0.001

    drug_input = layers.Input(shape=(max_drug_len,), name='drug_input', dtype='int32')
    drug_embedding = layers.Embedding(
        input_dim=drug_vocab_size,
        output_dim=EMBEDDING_DIM,
        name='drug_embedding'
    )(drug_input)
    drug_flat = layers.Flatten(name='drug_flatten')(drug_embedding)

    prot_input = layers.Input(shape=(max_prot_len,), name='protein_input', dtype='int32')
    prot_embedding = layers.Embedding(
        input_dim=prot_vocab_size,
        output_dim=EMBEDDING_DIM,
        name='protein_embedding'
    )(prot_input)
    prot_flat = layers.Flatten(name='protein_flatten')(prot_embedding)

    combined = layers.Concatenate(axis=1, name='concatenate_branches')([drug_flat, prot_flat])

    fc1 = layers.Dense(DENSE_UNITS_1, activation='relu', name='fully_connected_1')(combined)
    bn1 = layers.BatchNormalization(name='batch_norm_1')(fc1)
    drop1 = layers.Dropout(DROPOUT_RATE, name='dropout_1')(bn1)

    fc2 = layers.Dense(DENSE_UNITS_2, activation='relu', name='fully_connected_2')(drop1)
    bn2 = layers.BatchNormalization(name='batch_norm_2')(fc2)
    drop2 = layers.Dropout(DROPOUT_RATE, name='dropout_2')(bn2)

    output = layers.Dense(1, activation='linear', name='output')(drop2)

    model = Model(inputs=[drug_input, prot_input], outputs=output, name='Simplified_Dense_DTA')
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

print("All functions and classes defined successfully!")
print("OptimizedKibaDataProcessor is now available!")
print("Ready to process data!")


All functions and classes defined successfully!
OptimizedKibaDataProcessor is now available!
Ready to process data!


In [2]:
#Section 2

import zipfile
import os

# Define the path to your zip file
zip_file_path = '/content/data.zip' # Replace with the actual path to your .zip file

# Define the directory where you want to extract the contents
# If the directory doesn't exist, it will be created.
extract_dir = '/content/' # You can change this to your desired extraction path

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the zip file
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Successfully extracted '{zip_file_path}' to '{extract_dir}'")
except FileNotFoundError:
    print(f"Error: The file '{zip_file_path}' was not found.")
except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Optional: List the contents of the extracted directory to verify
print("\nContents of the extracted directory:")
for item in os.listdir(extract_dir):
    print(os.path.join(extract_dir, item))

Successfully extracted '/content/data.zip' to '/content/'

Contents of the extracted directory:
/content/.config
/content/data.zip
/content/kiba
/content/New Data
/content/.ipynb_checkpoints
/content/davis
/content/sample_data


In [3]:
#Section 3

# Process the data
print("=== Starting Data Processing ===")

# Check if data path exists
if not os.path.exists(KIBA_DATA_PATH):
    print(f"Data path not found: {KIBA_DATA_PATH}")
    print("Please make sure you have uploaded and extracted the kiba folder.")
    print("Current directory contents:")
    for item in os.listdir('.'):
        print(f"  - {item}")
else:
    print(f"Data path found: {KIBA_DATA_PATH}")

    # Now OptimizedKibaDataProcessor should be available
    processor = OptimizedKibaDataProcessor(data_path=KIBA_DATA_PATH)

    processed_data = processor.process_data()
    if processed_data is None:
        print("Failed to process data. Please check your data files.")
    else:
        (XD_train, XT_train, y_train), (XD_test, XT_test, y_test), (drug_vocab_size, prot_vocab_size) = processed_data
        print("Data processing completed successfully!")
        print(f"Data shapes:")
        print(f"   XD_train: {XD_train.shape}, dtype: {XD_train.dtype}")
        print(f"   XT_train: {XT_train.shape}, dtype: {XT_train.dtype}")
        print(f"   y_train: {y_train.shape}, dtype: {y_train.dtype}")
        print(f"   XD_test: {XD_test.shape}, dtype: {XD_test.dtype}")
        print(f"   XT_test: {XT_test.shape}, dtype: {XT_test.dtype}")
        print(f"   y_test: {y_test.shape}, dtype: {y_test.dtype}")


=== Starting Data Processing ===
Data path found: ./kiba


Chunk 1/10:   0%|          | 0/10000 [00:00<?, ?sample/s]
                                                         
Encoding drugs (chunk 1): 100%|██████████| 10000/10000 [00:00<00:00, 109997.22seq/s]

Chunk 2/10:   0%|          | 0/10000 [00:00<?, ?sample/s]
                                                         
Encoding drugs (chunk 2): 100%|██████████| 10000/10000 [00:00<00:00, 247897.63seq/s]

Chunk 3/10:   0%|          | 0/10000 [00:00<?, ?sample/s]
                                                         
Encoding drugs (chunk 3): 100%|██████████| 10000/10000 [00:00<00:00, 230251.31seq/s]

Chunk 4/10:   0%|          | 0/10000 [00:00<?, ?sample/s]
                                                         
Encoding drugs (chunk 4): 100%|██████████| 10000/10000 [00:00<00:00, 241072.74seq/s]

Chunk 5/10:   0%|          | 0/10000 [00:00<?, ?sample/s]
                                                         
Encoding drugs (chunk 5): 100%|██████████| 10000/10000 [00:00<00:00, 213771.

Data processing completed successfully!
Data shapes:
   XD_train: (94603, 100), dtype: int32
   XT_train: (94603, 1000), dtype: int32
   y_train: (94603,), dtype: float32
   XD_test: (23651, 100), dtype: int32
   XT_test: (23651, 1000), dtype: int32
   y_test: (23651,), dtype: float32


In [4]:
#Section 4

# CORRECTED CONFIGURATION
REDUCE_DATASET = True
DATASET_FRACTION = 1
EPOCHS = 100
BATCH_SIZE = 256  # REDUCED for better stability
VALIDATION_SPLIT = 0.1

print("CORRECTED CONFIGURATION:")
print("=" * 50)
print(f"Batch size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")

# FIXED: Proper dataset reduction
if 'XD_train' in locals() and len(XD_train) > 1000:  # Only if we have full data

    # Use the ORIGINAL full dataset, not the already reduced one
    print(f"Using original dataset: {len(XD_train):,} training samples")

    # Calculate correct sample sizes
    n_train = int(len(XD_train) * DATASET_FRACTION)
    n_test = int(len(XD_test) * DATASET_FRACTION)

    print(f"Target training samples: {n_train:,}")
    print(f"Target test samples: {n_test:,}")

    # Randomly sample subset
    np.random.seed(42)  # For reproducibility
    train_indices = np.random.choice(len(XD_train), n_train, replace=False)
    test_indices = np.random.choice(len(XD_test), n_test, replace=False)

    XD_train = XD_train[train_indices]
    XT_train = XT_train[train_indices]
    y_train = y_train[train_indices]

    XD_test = XD_test[test_indices]
    XT_test = XT_test[test_indices]
    y_test = y_test[test_indices]

    # Calculate batches per epoch
    batches_per_epoch = len(XD_train) // BATCH_SIZE

    print(f"CORRECTED DATASET:")
    print(f"   Training: {len(XD_train):,} samples")
    print(f"   Test: {len(XD_test):,} samples")
    print(f"   Batches per epoch: {batches_per_epoch}")
    print(f"   Much more stable training expected!")

    # Use corrected datasets
    XD_train = XD_train
    XT_train = XT_train
    y_train = y_train
    XD_test = XD_test
    XT_test = XT_test
    y_test = y_test

    print("Ready for corrected training!")

elif 'XD_train' in locals():
    print(f"Current dataset already reduced: {len(XD_train):,} samples")
    print("Please restart and run data processing again for full dataset")

    # Still apply batch size fix for current data
    batches_per_epoch = len(XD_train) // BATCH_SIZE
    print(f"Batches per epoch with current data: {batches_per_epoch}")

else:
    print("Cannot apply optimizations - data not loaded yet!")


CORRECTED CONFIGURATION:
Batch size: 256
Epochs: 100
Using original dataset: 94,603 training samples
Target training samples: 94,603
Target test samples: 23,651
CORRECTED DATASET:
   Training: 94,603 samples
   Test: 23,651 samples
   Batches per epoch: 369
   Much more stable training expected!
Ready for corrected training!


In [5]:
#Section 5

# Build the model (only if data processing was successful)
if 'XD_train' in locals() and 'drug_vocab_size' in locals():
    print("\n=== Building Optimized Model ===")

    # Build the fast model
    fast_model = build_improved_deepdta_model(
        drug_vocab_size=drug_vocab_size,
        prot_vocab_size=prot_vocab_size,
        max_drug_len=processor.max_drug_len,
        max_prot_len=processor.max_prot_len
    )

    print("\n=== Model Summary ===")
    fast_model.summary()
    print("Model built successfully!")
else:
    print("Cannot build model - data processing failed or incomplete")





=== Building Optimized Model ===

=== Model Summary ===


Model: "Simplified_Dense_DTA"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ drug_input          │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ protein_input       │ (None, 1000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ drug_embedding      │ (None, 100, 128)  │      3,840 │ drug_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ protein_embedding   │ (None, 1000, 128) │      2,688 │ protein_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ drug_flatten        │ (None, 12800)     │          0 │ drug_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ protein_flatten     │ (None, 128000)    │          0 │ protein_embeddin… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_branch… │ (None, 140800)    │          0 │ drug_flatten[0][… │
│ (Concatenate)       │                   │            │ protein_flatten[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fully_connected_1   │ (None, 1024)      │ 144,180,2… │ concatenate_bran… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm_1        │ (None, 1024)      │      4,096 │ fully_connected_… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 1024)      │          0 │ batch_norm_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fully_connected_2   │ (None, 512)       │    524,800 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm_2        │ (None, 512)       │      2,048 │ fully_connected_… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 512)       │          0 │ batch_norm_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │        513 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 144,718,209 (552.06 MB)

 Trainable params: 144,715,137 (552.04 MB)

 Non-trainable params: 3,072 (12.00 KB)

Model built successfully!


In [6]:
#Section 6

# OPTIMIZED TRAINING
if 'fast_model' in locals():
    print("STARTING OPTIMIZED TRAINING ...")
    print("=" * 55)

    # Training configuration summary
    print(f"Configuration Summary:")
    print(f"   Batch size: {BATCH_SIZE}")
    print(f"   Epochs: {EPOCHS}")
    print(f"   Training samples: {len(XD_train):,}")
    print(f"   Test samples: {len(XD_test):,}")
    print(f"   Batches per epoch: {len(XD_train) // BATCH_SIZE}")

    # Optimized callbacks for the full dataset
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,  # More patience for larger dataset
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,   # More patience for larger dataset
            min_lr=1e-6,
            verbose=1
        )
    ]

    print(f"\nStarting training...")
    print(f"Expected CI: 0.72-0.78 (significant improvement over 50%)")
    print(f"Expected total time: 1-3 minutes")
    print("=" * 55)

    import time
    start_time = time.time()

    # Train the model
    history = fast_model.fit(
        x=[XD_train, XT_train],
        y=y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=VALIDATION_SPLIT,
        callbacks=callbacks,
        verbose=1
    )

    end_time = time.time()
    training_time = end_time - start_time

    print(f"\nTRAINING COMPLETED!")
    print("=" * 30)
    print(f"Total training time: {training_time/60:.1f} minutes")
    print(f"Average per epoch: {training_time/len(history.history['loss']):.1f} seconds")
    print(f"Best validation loss: {min(history.history['val_loss']):.4f}")
    print(f"Final training loss: {history.history['loss'][-1]:.4f}")
    print(f"Total epochs completed: {len(history.history['loss'])}")

else:
    print("Cannot start training - model not built")

STARTING OPTIMIZED TRAINING ...
Configuration Summary:
   Batch size: 256
   Epochs: 100
   Training samples: 94,603
   Test samples: 23,651
   Batches per epoch: 369

Starting training...
Expected CI: 0.72-0.78 (significant improvement over 50%)
Expected total time: 1-3 minutes
Epoch 1/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 44s 107ms/step - loss: 74.9616 - mean_squared_error: 74.9616 - val_loss: 1.4548 - val_mean_squared_error: 1.4548 - learning_rate: 0.0010
Epoch 2/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 70s 89ms/step - loss: 3.0712 - mean_squared_error: 3.0712 - val_loss: 0.7100 - val_mean_squared_error: 0.7100 - learning_rate: 0.0010
Epoch 3/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 40s 86ms/step - loss: 2.2826 - mean_squared_error: 2.2826 - val_loss: 0.5736 - val_mean_squared_error: 0.5736 - learning_rate: 0.0010
Epoch 4/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 30s 90ms/step - loss: 1.7775 - mean_squared_error: 1.7775 - val_loss: 0.4940 - val_mean_squared_error: 0.4940 - learning_rate: 0.0010
Epoch 5/100
333/3

In [7]:
#Section 7

# EVALUATION
if 'history' in locals():
    print("EVALUATING RESULTS...")
    print("=" * 45)

    # Make predictions
    print("Making predictions on test set...")
    y_pred_test = fast_model.predict([XD_test, XT_test], batch_size=BATCH_SIZE, verbose=1)

    # Calculate metrics
    print("Computing metrics...")
    test_mse = mean_squared_error(y_test, y_pred_test)
    test_ci = concordance_index(y_test, y_pred_test)

    print(f"\n100% DATASET RESULTS:")
    print("=" * 30)
    print(f"Test MSE: {test_mse:.6f}")
    print(f"Test CI: {test_ci:.6f}")

    # Performance comparison across all datasets
    print(f"\nPERFORMANCE PROGRESSION:")
    print("-" * 35)
    print(f"   10% dataset: CI = 0.6552")
    print(f"   Full dataset: CI = {test_ci:.4f}")

    # Comparison with literature
    print(f"\nLiterature Comparison:")
    print("-" * 25)
    print(f"   Paper DeepDTA: 0.863")
    print(f"   Our 100% model: {test_ci:.3f}")
    print(f"   Performance ratio: {test_ci/0.863:.1%}")

    # Performance analysis
    if test_ci >= 0.72:
        print(f"\nEXCELLENT! CI ≥ 0.72 - Ready for full dataset!")
        print(f"Next step: Try Full dataset for CI ≈ 0.80+")
    elif test_ci >= 0.68:
        print(f"\nGOOD! CI ≥ 0.68 - Model scaling properly")
        print(f"Continue scaling to Full dataset")
    else:
        print(f"\nCI < 0.68 - May need model architecture improvements")

    # Additional metrics
    mae = np.mean(np.abs(y_test - y_pred_test.flatten()))
    rmse = np.sqrt(test_mse)
    r2 = 1 - np.var(y_test - y_pred_test.flatten()) / np.var(y_test)

    print(f"\nAdditional Metrics:")
    print("-" * 20)
    print(f"MAE: {mae:.6f}")
    print(f"RMSE: {rmse:.6f}")
    print(f"R² Score: {r2:.6f}")

    print("\n" + "="*45)
    print("Full DATASET EVALUATION COMPLETED!")
    print("="*45)

else:
    print("Cannot evaluate - training not completed")

EVALUATING RESULTS...
Making predictions on test set...
93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step
Computing metrics...

100% DATASET RESULTS:
Test MSE: 0.233471
Test CI: 0.842892

PERFORMANCE PROGRESSION:
-----------------------------------
   10% dataset: CI = 0.6552
   Full dataset: CI = 0.8429

Literature Comparison:
-------------------------
   Paper DeepDTA: 0.863
   Our 100% model: 0.843
   Performance ratio: 97.7%

EXCELLENT! CI ≥ 0.72 - Ready for full dataset!
Next step: Try Full dataset for CI ≈ 0.80+

Additional Metrics:
--------------------
MAE: 0.284383
RMSE: 0.483189
R² Score: 0.664987

Full DATASET EVALUATION COMPLETED!
